In [1]:
import pandas as pd
import os
import sys
sys.path.insert(0, "../")
from factor_cal.config_loader import basic_config as cfg

In [18]:
# read config file
config = cfg.BasicConfig('config/config.yml')
fac_names = []
for facType in config['factors']:
    fac_names += list(config['factors'][facType].keys())

In [24]:
# ic_results = pd.DataFrame({c: pd.Series(dtype=t) for c, t in {'IC Mean': 'float', 'IC Std.': 'float', 'Risk-Adjusted IC': 'float', 'IC_win rate': 'float'}.items()})

In [7]:
base_dir = r'C:\Users\12552\Downloads\ic_summary\factor_ic_summary'

In [39]:
ic_results = None
pred_type = '1m'
for fac_name in fac_names:
    cur_res = pd.read_csv(os.path.join(base_dir, f'{fac_name}.csv'), index_col=0)
    cur_res = cur_res.loc[pred_type, ['IC Mean', 'IC Std.', 'Risk-Adjusted IC', 'IC win rate']]
    cur_df = pd.DataFrame(cur_res).T
    cur_df['factor'] = fac_name
    ic_results = pd.concat([ic_results, cur_df], axis=0, ignore_index=True)

In [ ]:
# ic_results[abs(ic_results['Risk-Adjusted IC']) > 1.5].sort_values('IC Mean')

In [48]:
xlsx_file = os.path.join(base_dir, f'ic_summary_{pred_type}.xlsx')
ic_results.to_excel(xlsx_file, index=False)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\12552\\Downloads\\ic_summary\\factor_ic_summary\\ic_summary_1m.xlsx'

In [51]:
from openpyxl import load_workbook
from openpyxl.styles import Font

df = pd.read_excel(xlsx_file, sheet_name='Sheet1')
book = load_workbook(xlsx_file)
sheet = book['Sheet1']

In [54]:
for row in sheet.iter_rows():
    print(row)
    
    for cell in row:
        if cell.value == 'factor':
            cell.font = Font(bold=True)

(<Cell 'Sheet1'.A1>, <Cell 'Sheet1'.B1>, <Cell 'Sheet1'.C1>, <Cell 'Sheet1'.D1>, <Cell 'Sheet1'.E1>)
<Cell 'Sheet1'.A1>
<Cell 'Sheet1'.B1>
<Cell 'Sheet1'.C1>
<Cell 'Sheet1'.D1>
<Cell 'Sheet1'.E1>
(<Cell 'Sheet1'.A2>, <Cell 'Sheet1'.B2>, <Cell 'Sheet1'.C2>, <Cell 'Sheet1'.D2>, <Cell 'Sheet1'.E2>)
<Cell 'Sheet1'.A2>
<Cell 'Sheet1'.B2>
<Cell 'Sheet1'.C2>
<Cell 'Sheet1'.D2>
<Cell 'Sheet1'.E2>
(<Cell 'Sheet1'.A3>, <Cell 'Sheet1'.B3>, <Cell 'Sheet1'.C3>, <Cell 'Sheet1'.D3>, <Cell 'Sheet1'.E3>)
<Cell 'Sheet1'.A3>
<Cell 'Sheet1'.B3>
<Cell 'Sheet1'.C3>
<Cell 'Sheet1'.D3>
<Cell 'Sheet1'.E3>
(<Cell 'Sheet1'.A4>, <Cell 'Sheet1'.B4>, <Cell 'Sheet1'.C4>, <Cell 'Sheet1'.D4>, <Cell 'Sheet1'.E4>)
<Cell 'Sheet1'.A4>
<Cell 'Sheet1'.B4>
<Cell 'Sheet1'.C4>
<Cell 'Sheet1'.D4>
<Cell 'Sheet1'.E4>
(<Cell 'Sheet1'.A5>, <Cell 'Sheet1'.B5>, <Cell 'Sheet1'.C5>, <Cell 'Sheet1'.D5>, <Cell 'Sheet1'.E5>)
<Cell 'Sheet1'.A5>
<Cell 'Sheet1'.B5>
<Cell 'Sheet1'.C5>
<Cell 'Sheet1'.D5>
<Cell 'Sheet1'.E5>
(<Cell 'Sheet1'.A6>,

In [80]:
class Condition:
    def __init__(self, name, value, func):
        self._name = name
        self._value = value
        self._func = func
        
    @property
    def name(self):
        return self._name
    
    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self, value):
        self._value = value
    
    @property
    def index(self):
        return self._index
    
    @index.setter
    def index(self, index):
        self._index = index
    
    def is_satisfied(self, value):
        return self._func(value, self._value)

def abs_cond(value, threshold):
    return abs(value) > threshold

def win_rate_cond(value, threshold):
    return abs(value-0.5) > threshold

def determine_index(dic, conditions, name, index):
    for condition in conditions:
        if name == condition._name:
            condition.index = index
            dic[index] = condition

fac_index = None
satisfied_factors = []
conditions = [Condition('IC Mean', 0.03, abs_cond), Condition('Risk-Adjusted IC', 1.5, abs_cond), Condition('IC win rate', 0.3, win_rate_cond)]
cond_ind_dict = {}
for i, row in enumerate(sheet.iter_rows()):
    if i == 0:
        for j, cell in enumerate(row):
            if cell.value == 'factor':
                fac_index = j
                continue
            determine_index(cond_ind_dict, conditions, cell.value, j)
    else:
        satisfied_cond_num = 0
        factor_name = None
        for j, cell in enumerate(row):
            if j == fac_index:
                factor_name = cell.value
                continue
            if j in cond_ind_dict.keys():
                condition = cond_ind_dict[j]
                if condition.is_satisfied(cell.value):
                    satisfied_cond_num += 1
                    cell.font = Font(color='FF0000')
            
        if satisfied_cond_num >= 2:
            satisfied_factors.append(factor_name)
            
            
xlsx_file = os.path.join(base_dir, f'ic_summary_{pred_type}_modified.xlsx')                  
book.save(xlsx_file)

In [82]:
import yaml
yaml.dump(satisfied_factors, open(os.path.join(base_dir, f'satisfied_factors_{pred_type}.yml'), 'w'))

In [84]:
pred_type='1m'
base_dir = r'C:\Users\12552\Downloads\ic_summary\factor_ic_summary'
factor_filepath = os.path.join(base_dir, f'satisfied_factors_{pred_type}.yml')
with open(factor_filepath, 'r') as f:
    factor_names = yaml.load(f, Loader=yaml.FullLoader)

In [85]:
factor_names

['close_ret',
 'td_ret_v_prod_5min',
 'td_p_v_ratio_3s_1min',
 'td_p_v_ratio_3s_5min',
 'close_ret_15min',
 'close_ret_5min',
 'close_ret_1min',
 'ohlc_rat',
 'clh',
 'clh_delta',
 'td_buy_rank',
 'td_sell_rank',
 'close_adjusted',
 'retail_dir_tsrank_10min',
 'retail_dir_rowrank',
 'wb1_tsrank',
 'wb5_tsrank',
 'wb10_tsrank',
 'td_v_ratio',
 'bs_v1_tsrank',
 'bs_v5_tsrank',
 'bs_pv1_tsrank',
 'bs_pv5_tsrank',
 'en_b_price_tsrank',
 'en_s_price_tsrank',
 'bs_press',
 'en_v_order_inbalance']